In [ ]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
from scipy.spatial import cKDTree

In [5]:
# Convert time columns to numeric (seconds since epoch)
def to_epoch_seconds(dt_series):
    return pd.to_datetime(dt_series).astype('int64') // 10**9

def merge_nearest_4d(df1, df2, vars_to_add):
    # Ensure columns exist and are numeric
    for col in ['latitude', 'longitude', 'depth']:
        gfd_csv[col] = pd.to_numeric(df1[col], errors='coerce')
        df_nc[col] = pd.to_numeric(df2[col], errors='coerce')

    gfd_csv['time'] = to_epoch_seconds(gfd_csv['time'])
    df_nc['time'] = to_epoch_seconds(df_nc['time'])

    # Build KDTree for df2
    tree = cKDTree(df2[['latitude', 'longitude', 'depth', 'time_num']].values)
    # Query nearest for all df1 points
    distances, idxs = tree.query(df1[['latitude', 'longitude', 'depth', 'time_num']].values, k=1)
    # Extract vars from df2 for matched rows
    matched = df2.iloc[idxs][vars_to_add].reset_index(drop=True)
    # Concatenate results
    result = pd.concat([df1.reset_index(drop=True), matched], axis=1)
    return result

# Example usage:
# df1 = ...  # your main dataframe
# df2 = ...  # dataframe with variables to add
# vars_to_add = ['thetao', 'so', 'bottomT']  # columns from df2 to bring into df1

# merged_df = merge_nearest_4d(df1, df2, vars_to_add)